In [131]:
from tkinter import *
import time

root=Tk()
root.geometry("400x400")
#funct
def funct():
    for i in range(len(first_try)):
        for j in range(16):
            row=int(j/4)
            col=int(j%4)
            if row==0:
                    labels_0[col].config(bg=getColor(array[i][j]),text="{:.4f}".format(array[i][j]))
            elif row==1:
                    labels_1[col].config(bg=getColor(array[i][j]),text="{:.4f}".format(array[i][j]))
            elif row==2:
                    labels_2[col].config(bg=getColor(array[i][j]),text="{:.4f}".format(array[i][j]))
            elif row==3:
                    labels_3[col].config(bg=getColor(array[i][j]),text="{:.4f}".format(array[i][j]))
            
            root.update()
        root.after(1500)
            
       
       
            
      
        
#labels
labels_0=[]
labels_1=[]
labels_2=[]
labels_3=[]
#putting the sensors
for i in range(4):
    lab=Label(root,width="12",height="5",text="sensor_"+str(i),bg="white",borderwidth=1, relief="solid")
    lab.grid(row=0,column=i)
    labels_0.append(lab)
for i in range(4):
    lab=Label(root,width="12",height="5",text="sensor_"+str(i+4),bg="white",borderwidth=1, relief="solid")
    lab.grid(row=1,column=i)
    labels_1.append(lab)
for i in range(4):
    lab=Label(root,width="12",height="5",text="sensor_"+str(i+8),bg="white",borderwidth=1, relief="solid")
    lab.grid(row=2,column=i)
    labels_2.append(lab)
for i in range(4):
    lab=Label(root,width="12",height="5",text="sensor_"+str(i+12),bg="white",borderwidth=1, relief="solid")
    lab.grid(row=3,column=i)
    labels_3.append(lab)

#button
but=Button(root,text="click",command=funct)
but.grid(row=4,column=1)
root.mainloop()

In [111]:
#random array
from random import seed
from random import random
import numpy as np
# seed random number generator
seed(1)
array=np.zeros((30, 16))
for i in range (30):
    for j in range(16):
        value = random()
        array[i][j]=value
        
    


In [106]:
#color array
color=["#de4118","#e1542f","#e46646","#e77a5d",
       "#eb8d74","#eea08b","#f1b3a2","#f5c6b9",
       "#f8d9d0","#fbece7","#ffffff"
      ]
#color generator function
def getColor(x):
    if x==0:
        return color[10]
    elif x>0 and x<=0.1:
        return color[9]
    elif x>0.1 and x<=0.2:
        return color[8]
    elif x>0.2 and x<=0.3:
        return color[7]
    elif x>0.3 and x<=0.4:
        return color[6]
    elif x>0.4 and x<=0.5:
        return color[5]
    elif x>0.5 and x<=0.6:
        return color[4]
    elif x>0.6 and x<=0.7:
        return color[3]
    elif x>0.7 and x<=0.8:
        return color[2]
    elif x>0.8 and x<=0.9:
        return color[1]
    elif x>0.9 and x<=1:
        return color[0]
        
    

In [22]:
import numpy as np
a=np.array([[1,2,3], [ 4,5,6],[7, 8,9],[ 4,5,6],[ 10,5,6]])
trans=a.transpose()
max_arr=np.zeros(3)
min_arr=np.zeros(3)
for i in range(len(trans)):
    max_arr[i]=np.max(trans[i])
    min_arr[i]=np.min(trans[i])
print(a)
min_arr

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [ 4  5  6]
 [10  5  6]]


array([1., 2., 3.])

In [26]:
x=a-min_arr
x

array([[0., 0., 0.],
       [3., 3., 3.],
       [6., 6., 6.],
       [3., 3., 3.],
       [9., 3., 3.]])

array([[0.        , 0.        , 0.        ],
       [0.33333333, 0.5       , 0.5       ],
       [0.66666667, 1.        , 1.        ],
       [0.33333333, 0.5       , 0.5       ],
       [1.        , 0.5       , 0.5       ]])

In [ ]:
#!/usr/bin/env python
# coding: utf-8


import pandas as pd 
import numpy as np
from scipy.signal import medfilt,butter,filtfilt, savgol_filter
import matplotlib.pyplot as plt
from scipy import fftpack


def read_data(path):
    data = pd.read_csv(path,sep=';')
    #drop unneccessery data
    data=data.drop(['ts', 'D'], axis=1)
    arr=data.to_numpy()
    return arr


 
def normalize_data_simple(arr):
    min_arr=np.zeros(16)
    for i in range(arr.shape[1]):       
        min_arr[i]=np.min(arr[:,i])
        
    
    arr=arr-min_arr
    arr = arr/np.max(arr[50:])
    arr[arr > 0.5] = 1
    arr[arr <= 0.5] = 0
    return arr

def filter_arr(arr):
    arr[arr > 0.5] = 1
    arr[arr <= 0.5] = 0
    return arr
    


def normalize_data_conv(arr):
    
    min_arr=np.zeros(16)
    for i in range(arr.shape[1]):
        arr[:,i] = medfilt(arr[:,i])
        
    for i in range(arr.shape[1]):       
        min_arr[i]=np.min(arr[:,i])
        
    
    arr=arr-min_arr
    arr = arr/np.max(arr)
    #arr_filtered = lowPassFilterWithGain(arr[:,0], 40, 28.5 , 0)
    
    
    for i in range(arr.shape[1]):
        #arr[:,i] = np.convolve(arr[:,i], [0.2, 0.4, 0.6, 0.8, 1, 0.8, 0.6, 0.4, 0.2], 'same')
        arr[:,i] = np.convolve(arr[:,i], np.ones(30), 'same')
    
    arr = arr / np.max(arr) 
    return arr

def normalize_data_fft(arr):
    
    for i in range(arr.shape[1]):       
        
        sig=arr[:,i]
        time_step=0.035
        period = 3.
        sig_fft = fftpack.fft(sig)
        
        # And the power (sig_fft is of complex dtype)
        power = np.abs(sig_fft)
        
        # The corresponding frequencies
        sample_freq = fftpack.fftfreq(sig.size, d=time_step)
        
        pos_mask = np.where(sample_freq > 0)
        freqs = sample_freq[pos_mask]
        peak_freq = freqs[power[pos_mask].argmax()]
        
        # Check that it does indeed correspond to the frequency that we generate
        # the signal with
        np.allclose(peak_freq, 1./period)
        
        high_freq_fft = sig_fft.copy()
        high_freq_fft[np.abs(sample_freq) > 0.3] = 0
        filtered_sig = fftpack.ifft(high_freq_fft)
        arr[:,i] = filtered_sig
    
    
    min_arr=np.zeros(16)
    for i in range(arr.shape[1]):       
        min_arr[i]=np.min(arr[:,i])
        
    
    arr=arr-min_arr
    arr = arr/np.max(arr)
        
    return arr

#color generator function
def getColor(x):
    #color array
    color=["#de4118","#e1542f","#e46646","#e77a5d",
       "#eb8d74","#eea08b","#f1b3a2","#f5c6b9",
       "#f8d9d0","#fbece7","#ffffff"
      ]
    if x==0:
        return color[10]
    elif x>0 and x<=0.1:
        return color[9]
    elif x>0.1 and x<=0.2:
        return color[8]
    elif x>0.2 and x<=0.3:
        return color[7]
    elif x>0.3 and x<=0.4:
        return color[6]
    elif x>0.4 and x<=0.5:
        return color[5]
    elif x>0.5 and x<=0.6:
        return color[4]
    elif x>0.6 and x<=0.7:
        return color[3]
    elif x>0.7 and x<=0.8:
        return color[2]
    elif x>0.8 and x<=0.9:
        return color[1]
    elif x>0.9 and x<=1:
        return color[0]
        


# In[ ]:



from tkinter import *
import time
from tkinter import filedialog as fd 

root=Tk()
root.title("Mat Project")
root.geometry("1200x620")
global flag#for the stop operation
flag=False
#funct
def funct():
    global flag
    flag=False
    #file chooser
    name= fd.askopenfilename() 
    print("file path is ",name)
    #read the data
    f=read_data(name)
    #normalize the data
    
    
    name = name[:-4]
    array1 = read_data(name + "_T.csv")
    array1 = array1[5:]
    
    input_array = np.copy(f)
    input_array2=np.copy(f)
    array2 = normalize_data_fft(input_array[5:])
    array3 = normalize_data_conv(input_array2[5:])
    input_fft=np.copy(array2)
    array4=filter_arr(input_fft)
    input_conv=np.copy(array3)
    array5=filter_arr(input_conv)
    

    
    
    print(array1[0])
    
    for i in range(len(array1)):
       
        if flag==True:#if stop is true clear GUI
            for i in range(4):
                labels_0[i].config(bg="white",text="sensor_"+str(i))
                labels_1[i].config(bg="white",text="sensor_"+str(i+4))
                labels_2[i].config(bg="white",text="sensor_"+str(i+8))
                labels_3[i].config(bg="white",text="sensor_"+str(i+12))
                
                Blabels_0[i].config(bg="white",text="sensor_"+str(i))
                Blabels_1[i].config(bg="white",text="sensor_"+str(i+4))
                Blabels_2[i].config(bg="white",text="sensor_"+str(i+8))
                Blabels_3[i].config(bg="white",text="sensor_"+str(i+12))
                
                root.update()
                
            return#exit from the function
        #the color operation is being handled here
        for j in range(16):
            row=int(j/4)
            col=int(j%4)
            if row==0:
                    labels_0[col].config(bg=getColor(array1[i][j]))
                    Blabels_0[col].config(bg=getColor(array2[i][j]))
                    Clabels_0[col].config(bg=getColor(array3[i][j]))
                    Dlabels_0[col].config(bg=getColor(array4[i][j]))
                    Elabels_0[col].config(bg=getColor(array5[i][j]))
            elif row==1:
                    labels_1[col].config(bg=getColor(array1[i][j]))
                    Blabels_1[col].config(bg=getColor(array2[i][j]))
                    Clabels_1[col].config(bg=getColor(array3[i][j]))
                    Dlabels_1[col].config(bg=getColor(array4[i][j]))
                    Elabels_1[col].config(bg=getColor(array5[i][j]))
            elif row==2:
                    labels_2[col].config(bg=getColor(array1[i][j]))
                    Blabels_2[col].config(bg=getColor(array2[i][j]))
                    Clabels_2[col].config(bg=getColor(array3[i][j]))
                    Dlabels_2[col].config(bg=getColor(array4[i][j]))
                    Elabels_2[col].config(bg=getColor(array5[i][j]))
            elif row==3:
                    labels_3[col].config(bg=getColor(array1[i][j]))
                    Blabels_3[col].config(bg=getColor(array2[i][j]))
                    Clabels_3[col].config(bg=getColor(array3[i][j]))
                    Dlabels_3[col].config(bg=getColor(array4[i][j]))
                    Elabels_3[col].config(bg=getColor(array5[i][j]))
            
            root.update()
        root.after(34)

def stop():#just arrenges the stop flag
    global flag
    flag=True
    
        
#each label array is a row
labels_0=[]#row 1
labels_1=[]#row 2
labels_2=[]#row 3
labels_3=[]#row 4
#setting the each lablel

lab=Label(root,width="16",height="2",text="Binary Forms",bg="white",borderwidth=1, relief="solid")
lab.grid(row=5,column=13)

Dlabels_0=[]#row 1
Dlabels_1=[]#row 2
Dlabels_2=[]#row 3
Dlabels_3=[]#row 4
add=2
for i in range(4):
    #setting columns 
    lab=Label(root,width="10",height="4",text="(0,"+str(i)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=1+add,column=i)
    labels_0.append(lab)
    #2x1
    lab=Label(root,width="10",height="4",text="(0,"+str(i)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=6,column=i+9)
    Dlabels_0.append(lab)
    
      #setting columns 
    lab=Label(root,width="10",height="4",text="(1,"+str(i+4)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=2+add,column=i)
    labels_1.append(lab)
    #2x1
    lab=Label(root,width="10",height="4",text="(1,"+str(i+4)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=7,column=i+9)
    Dlabels_1.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(2,"+str(i+8)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=3+add,column=i)
    labels_2.append(lab)
    #2x1
    lab=Label(root,width="10",height="4",text="(2,"+str(i+8)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=8,column=i+9)
    Dlabels_2.append(lab)
     #setting columns 
    lab=Label(root,width="10",height="4",text="(3,"+str(i+12)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=4+add,column=i)
    labels_3.append(lab)
    #2x1
    lab=Label(root,width="10",height="4",text="(3,"+str(i+12)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=9,column=i+9)
    Dlabels_3.append(lab)



#second part 1x2
Blabels_0=[]#row 1
Blabels_1=[]#row 2
Blabels_2=[]#row 3
Blabels_3=[]#row 4



for i in range(4):
    #setting columns 
    lab=Label(root,width="10",height="4",text="(0,"+str(i)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=1,column=i+9)
    Blabels_0.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(1,"+str(i+4)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=2,column=i+9)
    Blabels_1.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(2,"+str(i+8)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=3,column=i+9)
    Blabels_2.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(3,"+str(i+12)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=4,column=i+9)
    Blabels_3.append(lab)
#lejant
lab=Label(root,width="10",height="1",text="0-10%",bg="#fbece7",borderwidth=1, relief="solid")
lab.grid(row=1,column=0)
lab=Label(root,width="10",height="1",text="20-40%",bg="#f5c6b9",borderwidth=1, relief="solid")
lab.grid(row=1,column=1)
lab=Label(root,width="10",height="1",text="40-60%",bg="#eea08b",borderwidth=1, relief="solid")
lab.grid(row=1,column=2)
lab=Label(root,width="10",height="1",text="60-80%",bg="#e46646",borderwidth=1, relief="solid")
lab.grid(row=1,column=3)
lab=Label(root,width="10",height="1",text="80-100%",bg="#de4118",borderwidth=1, relief="solid")
lab.grid(row=1,column=4)


#third part 1x3
Clabels_0=[]#row 1
Clabels_1=[]#row 2
Clabels_2=[]#row 3
Clabels_3=[]#row 4

#2x3
Elabels_0=[]#row 1
Elabels_1=[]#row 2
Elabels_2=[]#row 3
Elabels_3=[]#row 4


for i in range(4):
    #setting columns 
    lab=Label(root,width="10",height="4",text="(0,"+str(i)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=1,column=i+14)
    Clabels_0.append(lab)
    #2x3
    lab=Label(root,width="10",height="4",text="(0,"+str(i)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=6,column=i+14)
    Elabels_0.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(1,"+str(i+4)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=2,column=i+14)
    Clabels_1.append(lab)
    #2x3
    lab=Label(root,width="10",height="4",text="(1,"+str(i+4)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=7,column=i+14)
    Elabels_1.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(2,"+str(i+8)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=3,column=i+14)
    Clabels_2.append(lab)
    #2x3
    lab=Label(root,width="10",height="4",text="(2,"+str(i+8)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=8,column=i+14)
    Elabels_2.append(lab)
      #setting columns 
    lab=Label(root,width="10",height="4",text="(3,"+str(i+12)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=4,column=i+14)
    Clabels_3.append(lab)
    #2x3
    lab=Label(root,width="10",height="4",text="(3,"+str(i+12)+")",bg="white",borderwidth=1, relief="solid")
    lab.grid(row=9,column=i+14)
    Elabels_3.append(lab)





lab1=Label(root,width="10",height="1",text="FFT",bg="white",borderwidth=1, relief="solid")
lab1.grid(row=0,column=10)

lab1=Label(root,width="10",height="1",text="Conv",bg="white",borderwidth=1, relief="solid")
lab1.grid(row=0,column=15)

#button
but=Button(root,text="Choose File",command=funct,width="9")
but.grid(row=7,column=2)
root.mainloop()






In [68]:
aw=[[0.1,0,6],
    [0.5,0.2]]

aw[aw > 0.5] = 1
aw

TypeError: '>' not supported between instances of 'list' and 'float'